# For this user prediction model we will use a linear regression with some extra stuff
Let`s define the variables we will use as predictors and targets
-Age|, usage_of_platform|, popular_genre|, money_spent on platform|, number_of_subsriptions|
-usage_of_platform - defined on a 3 point scale low, medium, high. The usage is counted as the number of liked songs, playlists, users following|, artists following|, playlists_following|, 

In [175]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from datetime import datetime, date

In [109]:
# Load the data to be used 
df_users = pd.read_csv(r"/home/jirka/Documents/spotify/project/spotify/analysis/analysis/data/Users_data.csv")
df_songs = pd.read_csv(r"/home/jirka/Documents/spotify/project/spotify/analysis/analysis/data/Songs_data.csv")
df_artists = pd.read_csv(r"/home/jirka/Documents/spotify/project/spotify/analysis/analysis/data/Artists_data.csv")
df_playlists = pd.read_csv(r"/home/jirka/Documents/spotify/project/spotify/analysis/analysis/data/Playlists_data.csv")
df_playlists_following = pd.read_csv(r"/home/jirka/Documents/spotify/project/spotify/analysis/analysis/data/Playlists_users_data.csv")
df_likes = pd.read_csv(r"/home/jirka/Documents/spotify/project/spotify/analysis/analysis/data/Likes_data.csv")
df_artists_following = pd.read_csv(r"/home/jirka/Documents/spotify/project/spotify/analysis/analysis/data/Artists_followers_data.csv")
df_users_following = pd.read_csv(r"/home/jirka/Documents/spotify/project/spotify/analysis/analysis/data/Followers_users_data.csv")
df_users_subscriptions = pd.read_csv(r"/home/jirka/Documents/spotify/project/spotify/analysis/analysis/data/User_subscriptions_data.csv")
df_subscription_info = pd.read_csv(r"/home/jirka/Documents/spotify/project/spotify/analysis/analysis/data/Subscription_plan_info_data.csv")

In [33]:
# Define helper functions
def change_users_age(birthday_str):
    today_date = date.today()
    date_converted = datetime.strptime(birthday_str, "%Y-%m-%d").date()

    age = today_date.year - date_converted.year
    
    return age

# Data cleanup and manipulation

In [ ]:
# Data manipulation for usage_of_platform column
usage_of_platform = df_playlists_following.groupby(by='user_id')["playlists_id"].count().reset_index()
usage_of_platform.rename(columns={'playlists_id': 'engagement_count'}, inplace=True)

# Add engagement count from artists following
df_artists_following_count = df_artists_following.groupby(by='user_id')['artist_id'].count().reset_index()
usage_of_platform = usage_of_platform.merge(
    df_artists_following_count, 
    on='user_id', 
    how='left', 
    suffixes=('', '_artists')
)
usage_of_platform['engagement_count'] += usage_of_platform['artist_id'].fillna(0)

# Add engagement count from users following
users_following_count = df_users_following.groupby(by='user_id1')['user_id2'].count().reset_index()
usage_of_platform = usage_of_platform.merge(
    users_following_count.rename(columns={'user_id1': 'user_id'}), 
    on='user_id', 
    how='left', 
    suffixes=('', '_users')
)
usage_of_platform['engagement_count'] += usage_of_platform['user_id2'].fillna(0)

# Add engagement count from likes
likes_count = df_likes.groupby(by='user_id')['song_id'].count().reset_index()
usage_of_platform = usage_of_platform.merge(
    likes_count, 
    on='user_id', 
    how='left', 
    suffixes=('', '_likes')
)
usage_of_platform['engagement_count'] += usage_of_platform['song_id'].fillna(0)

# Add engagement count from playlists created
playlists_count = df_playlists.groupby(by='creator_id')['id'].count().reset_index()
usage_of_platform = usage_of_platform.merge(
    playlists_count.rename(columns={'creator_id': 'user_id'}), 
    on='user_id', 
    how='left', 
    suffixes=('', '_playlists')
)
usage_of_platform['engagement_count'] += usage_of_platform['id'].fillna(0)

# Drop temporary columns used in merging
columns_to_drop = ['artist_id', 'user_id2', 'song_id', 'id']
usage_of_platform.drop(columns=columns_to_drop, inplace=True)

bins = [0, 8, 17, float('inf')]  # Define bin edges: 0-50 (low), 50-100 (mid), >100 (high)
labels = ['low', 'mid', 'high']

usage_of_platform['enganment_bins'] = pd.cut(usage_of_platform['engagement_count'], bins=bins, labels=labels, right=False)

usage_of_platform


In [ ]:
# Data manipulation for price, number_of_resubscriptions, number_of_money_spent column

df_users_subs_info = df_users_subscriptions.merge(df_subscription_info, how='left', left_on='subscription_plan_id', right_on='id', suffixes=('_user_sub', 'sub'))
df_users_subs_info_grouped = df_users_subs_info.groupby(by='user_id')['price'].sum().reset_index()
print(df_users_subs_info_grouped)

df_users_subscriptions_grouped = df_users_subscriptions.groupby(by='user_id')['subscription_plan_id'].count().reset_index()


In [ ]:
# Data manipulation for favorite genre
df_songs_artists = df_songs.merge(df_artists, how='left', left_on='artist_id', right_on='id', suffixes=('_song', '_artist'))

df_likes_songs_artists = df_likes.merge(df_songs_artists, how='left', left_on='song_id', right_on='id_song')
df_likes_songs_artists_grouped = df_likes_songs_artists.groupby(by=['user_id', 'genre'])['song_id'].count().reset_index()

idx = df_likes_songs_artists_grouped.groupby('user_id')['song_id'].idxmax()

# Select the rows with the maximum song_count for each user_id
favorite_genre = df_likes_songs_artists_grouped.loc[idx].reset_index(drop=True)

In [ ]:
# Creating the final analysis df
Analysis_df = pd.DataFrame()
Analysis_df['User_id'] = df_users['id']
Analysis_df["Age"] =  df_users["date_of_birth"].apply(change_users_age)
Analysis_df['User_type'] = df_users['user_type']
Analysis_df['Enganment_bins'] = usage_of_platform['enganment_bins']
Analysis_df = Analysis_df.merge(df_users_subscriptions_grouped, how='left', left_on='User_id', right_on='user_id')
Analysis_df = Analysis_df.merge(df_users_subs_info_grouped, how='left', left_on='User_id', right_on='user_id')
Analysis_df.rename(columns={"subscription_plan_id": "subscription_count", 
                            "price": "money_spent"}, inplace=True)
Analysis_df.drop(columns=["user_id_x", "user_id_y"], inplace=True)
Analysis_df["Favorite_genre"] = favorite_genre["genre"]
Analysis_df = Analysis_df.loc[Analysis_df['User_type'] == 'premium']

Analysis_df.drop(columns=["User_type"], inplace=True)
Analysis_df
Analysis_df["resubscribed"] = np.where(Analysis_df['subscription_count'] > 1, 1, 0)

Analysis_df

,User_id,Age,Enganment_bins,subscription_count,money_spent,Favorite_genre,resubscribed
1,2,39,high,6.0,22494.0,Pop,1
2,3,32,high,4.0,23496.0,Jazz,1
3,4,36,high,4.0,53996.0,Pop,1
6,7,29,high,4.0,21496.0,Pop,1
7,8,37,high,4.0,45996.0,Hip-Hop,1
8,9,30,high,4.0,15996.0,Pop,1
10,11,35,high,2.0,21498.0,Hip-Hop,1
11,12,31,high,1.0,1499.0,Pop,0
13,14,38,high,2.0,19498.0,Pop,1
14,15,32,high,2.0,5998.0,Jazz,1


# Basic descriptives and descriptive graphs

In [ ]:
Analysis_df.describe()

plt.figure(figsize=(15, 12))
sns.histplot(Analysis_df["Age"], bins=10, kde=True)
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(Analysis_df["Enganment_bins"])
plt.title('Box Plot of Numerical Columns')
plt.show()

plt.figure(figsize=(10, 6))
sns.scatterplot(x='Age', y='money_spent', hue='Enganment_bins', data=Analysis_df)
plt.xlabel('Age')
plt.ylabel('Money_spent')
plt.title('Scatter plot of money_spent by age and enganmnet')

plt.figure(figsize=(10, 6))
sns.scatterplot(x='subscription_count', y='money_spent', hue='Enganment_bins', data=Analysis_df)
plt.xlabel('Subscription_count')
plt.ylabel('Money spent')
plt.title('Scatter plot of money spent by subscription count')

# Model creation

In [191]:
X = Analysis_df.drop(columns=["resubscribed"])
y = Analysis_df["resubscribed"]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ]
)

# Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=3)),
    ('regressor', Ridge())
])

param_grid = {
    'poly__degree': [1, 2, 3],  
    'regressor__alpha': [0.1, 1.0, 10.0]  
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring='neg_mean_squared_error', 
    cv=5,  
    n_jobs=-1  
)

grid_search.fit(x_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best MSE:", -grid_search.best_score_)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)
print("Training MSE:", mean_squared_error(y_test, y_pred))

Best Parameters: {'poly__degree': 2, 'regressor__alpha': 10.0}
Best MSE: 0.08242471736326833
Training MSE: 0.08321237628121009


# Conclusion
Building model using predictors to see if people will resubsribe yields good scores overall. 
Best MSE: 0.08242471736326833
Training MSE: 0.08321237628121009
This model might be used when app goes into production